<a href="https://colab.research.google.com/github/valmirf/redes_neurais_pos/blob/main/PyTorch/Introdu%C3%A7%C3%A3o_ao_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Frameworks

---
![](https://raw.githubusercontent.com/valmirf/redes_neurais_ple/master/PyTorch/FIG/frameworks.png)


#Empresas envolvidas

![](https://raw.githubusercontent.com/valmirf/redes_neurais_ple/master/PyTorch/FIG/frameworks2.png)


#PyTorch

PyTorch é um framework de Python que funciona como uma camada em cima do Tensorflow, facilitando seu uso e facilitando procedimentos no uso de redes neurais artificiais e GPU. PyTorch tem se tornado padrão em várias empresas e institutos de pesquisa.

Nesse tutorial, vamos aprender a usar o PyTorch no treinamento e utilização de uma rede neural artificial. O tutorial seguirá várias etapas, desde a leitura de uma base de dados, até o treinamento e avaliação da rede neural treinada. 



In [ ]:
#Import packages

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import numpy as np
import pickle
import os
from PIL import Image
import random
import time
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
import time


import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np

#Test if GPU Cuda is available
cuda_available = torch.cuda.is_available()
print(cuda_available)

###Nesse exercício, nós treinaremos uma rede MLP para classificar imagens da base MNIST.



# 1. Carregar uma base de dados no PyTorch

A biblioteca torchvision.datasets possui as principais bases de dados disponíveis para download no PyTorch (ver: https://pytorch.org/docs/stable/torchvision/datasets.html)

Algumas bases de dados:

*   MNIST
*   COCO
*   ImageNet
*   CIFAR
*   Flickr
*   VOC
*   Cityscapes



Iremos usar a função DataLoader importada de TorchVision para carregar a base de dados MNIST. Usaremos um lote (batch) de tamanho 64 para treinamento e tamanho 1000 para teste neste conjunto de dados. O módulo TorchVision oferece ainda, muitas transformações úteis, como corte ou normalização. Os valores 0.1307 e 0.3081 são usados para a função Normalize(). Esses valores correspondem a média global e o desvio padrão do conjunto de dados MNIST. 

Usando o DataLoader, também poderíamos usar num_workers> 1, para usar subprocessos para carregar dados de forma assíncrona ou usar uma parte da memória RAM fixa (via pin_memory), para acelerar as transferências de RAM para GPU. 

In [ ]:
batch_size=64
cuda=0


#normalize with mean and standard deviation
normalize = transforms.Normalize(mean=(0.1307,),std=(0.3081,))
transform = transforms.Compose([transforms.ToTensor(), normalize])

# define three datasets in order to have different transforms
# on training, validation and test
dataset_train = datasets.MNIST('/data/mnist/train', download=True, train=True, transform=transform)
#dataset_val = datasets.MNIST('/data/mnist/val', download=True, train=True, transform=transform)
dataset_test = datasets.MNIST('/data/mnist/test', download=True, train=False, transform=transform)

 
trainloader = torch.utils.data.DataLoader(dataset_train, batch_size, shuffle=True)
#valloader = torch.utils.data.DataLoader(dataset_val, batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_test, test_batch_size)

Agora, vamos dar uma olhada em alguns exemplos da base de dados de teste:

In [ ]:
examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)

Vamos ver em que consiste um lote de dados de teste. Portanto, um lote de dados do conjunto de teste é um tensor de forma:

In [ ]:
example_data.shape

Isso significa que temos 1000 exemplos de 28x28 pixels em escala de cinza (ou seja, sem canais RGB, portanto, um). Podemos plotar alguns deles usando o matplotlib.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Rótulo: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

Ver a imagem em mais detalhes:

In [ ]:
def visualize_input(img, ax): 
    #img = np.uint8(img)
    #print('img: ', img)
    max = img.max()
    min = img.min()
    data = (img-min) / (max-min) # normalize the data to 0 - 1
    data = 255 * data # Now scale by 255
    img = np.uint8(data)
    #img = np.uint8(img.mul(255).numpy())
    #print('img: ', np.uint8(img.numpy()))
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            #ax.annotate(str(round(img[x][y],2)), xy=(y,x),
            ax.annotate(str(img[x][y]), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
visualize_input(example_data[0][0], ax)

# 2. Definir o Modelo de Arquitetura

Nessa etapa, vamos definir o modelo da arquitetura da rede neural. O modelo criado terá uma camada de entrada, uma camada de saída de dez neurônios e duas camadas ocultas entre elas, uma com 128 e outra com 64 neurônios.

O módulo torch.nn permite constuir a arquitetura da rede de forma bastante simples. 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

## Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(512, 512)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(512, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        #self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

# initialize the NN
model = Net()
print(model)

O módulo nn.Sequential envolve as camadas na rede. Existem duas camadas lineares com a ativação da ReLU (uma função simples que permite a passagem de valores positivos, enquanto valores negativos são modificados para zero). A camada de saída é uma camada linear com a ativação da função LogSoftmax. Tecnicamente, uma função LogSoftmax é o logaritmo de uma função Softmax.

Além disso, temos 784 unidades na primeira camada, porque redimensionamos cada imagem antes de enviá-la para dentro da rede neural. (28 x 28 = 784)

In [ ]:
class Net2(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(Net2, self).__init__()
    self.seq = nn.Sequential(
               nn.Linear(input_dim,hidden_dim),
               nn.ReLU(),
               nn.Linear(hidden_dim,output_dim),
               nn.LogSoftmax(dim=1)
               )

  def forward(self, x):
    # flatten image input
    x = x.view(-1, 28 * 28)
    return self.seq(x)

# initialize the NN
#model = Net2(28*28,256,10)
#model = Net2()
#print(model)

# 3. Modelo de otimização

Em seguida, definimos a função de perda como a probabilidade de log negativa. Juntas, as funções LogSoftmax() e o NLLLoss(), atuam como a perda de entropia cruzada com SoftMax.


## 3.1 Define o otimizador (SGD)

[https://pytorch.org/optimizer](https://pytorch.org/cppdocs/api/classtorch_1_1optim_1_1_optimizer.html)

### Tipos de otimizadores:
*   SGD
*   RMSprop
*   Adagrad
*   Adam


## 3.2 Especifica função de perda (categorical_crossentropy)

[https://pytorch.org/nn/](https://pytorch.org/docs/stable/nn.html#)

### Tipos de funções de perda
*   mean_squared_error 
*   mean_absolute_error
*   mean_absolute_percentage_error
*   mean_squared_logarithmic_error
*   squared_hinge
*   hinge
*   categorical_hinge
*   logcosh
*   categorical_crossentropy
*   sparse_categorical_crossentropy
*   binary_crossentropy
*   kullback_leibler_divergence
*   poisson
*   cosine_proximity


## 3.3 Como escolher uma função de perda?

### Funções de perda para regressão
*   Mean Squared Error
*   Mean Squared Logarithmic Error
*   Mean Absolute Error

### Funções de perda para classificação binária
*   Binary Cross-Entropy
*   Hinge
*   Squared Hinge 

### Funções de perda para classificação multiclasse
*   Multi-Class Cross-Entropy
*   Sparse Multiclass Cross-Entropy
*   Categorical Hinge
*   Kullback Leibler Divergence


In [ ]:

## Specify loss and optimization functions

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# 4. Treinar o Modelo

Nessa etapa, o treinamento da rede neural é realizado. Sua rede neural repete o conjunto de treinamento e atualiza os pesos. Utilizamos o módulo torch.optim, que é um módulo do PyTorch para otimizar o modelo. Nessa arquitetura, usamos a descida de gradiente e os pesos são atualizados por backpropagation. Assim, em cada época (número de vezes que repetimos o conjunto de treinamento), veremos uma diminuição gradual do valor da função de perda.

Observe que para a fase de treinamento, o modelo está no modo de treinamento - (model.train()) e também precisamos zerar os gradientes de cada lote. 

In [ ]:
# number of epochs to train the model
n_epochs = 10  # suggest training between 20-50 epochs

model.train() # prep model for training

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in trainloader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # Flatten MNIST images into a 784 long vector
        #data = images.view(data.shape[0], -1)


        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
    else:
        # print training statistics 
        # calculate average loss over an epoch
        train_loss = train_loss/len(trainloader)
        print("Epoch {} - Training loss: {}".format(epoch, train_loss/len(trainloader)))
        #print("\nTraining Time (in minutes) =",(time()-time0)/60)        
     

# 5. Avaliando o modelo no conjunto de teste

Agora, o modelo treinado na etapa anterior será avaliado. 


In [ ]:
images, labels = next(iter(testloader))

img = images[0].view(1, 784)

with torch.no_grad():
  logps = model(img)

ps = torch.exp(logps)
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))
print("Digit =", labels.numpy()[0])

Ness etapa, iteramos o conjunto de teste e calculamos o número total de previsões corretas. 

Observe que precisamos especificar que estamos usando o modelo apenas para avaliação - (model.eval()). Não é preciso calcular os gradientes.

In [ ]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in testloader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

 ## 6. Salvando o modelo

Para salvar o modelo treinado, para que não seja preciso treinar novamente quando for usar o modelo, salvamos o modelo. Quando precisarmos no futuro, podemos carregá-lo e usá-lo diretamente, sem treinamento adicional.


In [ ]:
torch.save(model, './my_mnist_model.pt') 

#TESTES
1. Modifique a quantidade de camadas do modelo treinado
2. Modifique a quantidade de neurônios na(s) camada(s) intermediárias do modelo treinado
3. Modifique o tipo de otimizador do seu modelo
4. Modifique a função de perda do seu modelo
5. Avalie combinações entre tipos de otimizador, função de perda, tipos de camadas e quantidade de épocas no treinamento. Qual foi a melhor taxa de acerto no conjunto de teste? Qual a melhor configuração de modelo que você encontrou? 